## Jane Streek Market Price Prediction 📈

![image](https://storage.googleapis.com/kaggle-competitions/kaggle/23304/logos/thumb76_76.png?t=2020-11-16-17-41-20)

Today's Trading system is lot depended on use of technology, market is evolving everyday and inorder to survive this dynamic nature of market one needs to use all the tools available to him/her. <br/>

The use of machine learning in market price prediction has been increased as machine learning models are very good at understading patterns and making predictions.

Machine learning model along with human knowledge can help you to make proper market predictions.

## what is this competition about 💡?

Here the challange is to make a model which can predict wether to accept a trade(1) or reject a trade(0)<br/>
given the data. 

So it is a classification problem.<br/>
but not exactly as we can not submit probabilities.

## Importing Libraries 📗.

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

In [ ]:
import os
import gc
import sys
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
import datatable as dt
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from mlens.ensemble import SuperLearner

from sklearn.manifold import TSNE

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold, StratifiedKFold

from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Label,Range1d, Slider, Span
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# sys.path.append('../input/pytorchtabnet/')
# !pip install '../input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl'

# from pytorch_tabnet.tab_model import TabNetClassifier

## 1. Given Data 💽

We are provided with 4 files 

1. example_sample_submission.csv
2. features.csv
3. example_test.csv
4. train.csv

train.csv is the main file which contains feature_0 to feature_129 for each trade which is market data which we will use for making prediction and each trade has associated weight and resp which together represents a return on the trade.<br/>

It also contain date column for day of trade. <br/>
ts_id is for time ordering.<br/>

It also has resp_1,2,3,4 for returns over different time horizons, but resps are not present in test data.

features.csv contains metadata of features bu no information about metadata is provided.

example_test shows how private test data will look like.

example_sample_submission is example of how submission file should look like.

We have to use api provided by kaggle to make the prediction.

## 2. Metrics 📐

For each row in the test set we have to predict an action value 1 to make the trade and 0 to pass on it.

each trade has associated weight and resp score which will be used to calculate utility score which is the final score.

For each i

pi= ∑j(weightij∗respij∗actionij)

t= ∑pi/∑p**2i * √250|i|

i is no of unique dates in test set.

utitily = min(max(t,0),6)∑pi.

## 3. Loading Data and Stats💽

df.fread().to_pandas() is faster than pd.read_csv()

In [ ]:
%%time
folder_path = '../input/jane-street-market-prediction/'
train_data = dt.fread(folder_path + 'train.csv').to_pandas()
features = dt.fread(folder_path + 'features.csv').to_pandas()
sample = dt.fread(folder_path + 'example_sample_submission.csv').to_pandas()
test_data = dt.fread(folder_path + 'example_test.csv').to_pandas()

In [ ]:
print("{0}Number of rows in train data: {1}{2}\n{0}Number of columns in train data: {1}{3}".format(y_,r_,train_data.shape[0],train_data.shape[1]))
print("{0}Number of rows in test data: {1}{2}\n{0}Number of columns in test data: {1}{3}".format(m_,r_,test_data.shape[0],test_data.shape[1]))
print("{0}Number of rows in features data: {1}{2}\n{0}Number of columns in features data: {1}{3}".format(b_,r_,features.shape[0],features.shape[1]))
print("{0}Number of rows in sample data: {1}{2}\n{0}Number of columns in sample data: {1}{3}".format(c_,r_,sample.shape[0],sample.shape[1]))

In [ ]:
train_data.head()

In [ ]:
features.head()

In [ ]:
sample.head()

In [ ]:
test_data.head()

## 4. Exploratory Data Analysis 📊📈📉👀

### 4.1 Checking for Null Values

In [ ]:
print("Number of features with null values:",np.sum(train_data.isna().sum()>0))

### 4.1.1 Count of Null values in features 

zoom in to see the hidden features

In [ ]:
features = [f"feature_{x}" for x in range(130)]
null_count = train_data[features].isna().sum().sort_values(ascending=True)
null_count = null_count[null_count >0]
fig = px.bar(y=null_count.index,x=null_count.values)
fig.show()

There are too many features with null values right now we will just fill those values with mean values.

In [ ]:
train_data.fillna(train_data.mean(),inplace=True)
print("Number of features with null values:",np.sum(train_data.isna().sum()>0))

### 4.2 Distribution of weights 

In [ ]:
def distribution1(feature,color,df=train_data):
    plt.figure(figsize=(15,7))
    plt.subplot(121)
    sns.distplot(df[feature],color=color)
    plt.subplot(122)
    sns.violinplot(df[feature])
    print("{}Max value of {} is: {} {:.2f} \n{}Min value of {} is: {} {:.2f}\n{}Mean of {} is: {}{:.2f}\n{}Standard Deviation of {} is:{}{:.2f}"\
      .format(y_,feature,r_,df[feature].max(),g_,feature,r_,df[feature].min(),b_,feature,r_,df[feature].mean(),m_,feature,r_,df[feature].std()))

In [ ]:
distribution1('weight',color='yellow')

In [ ]:
print("Number of rows with weight zero:",np.sum(train_data.weight==0))
print("Percentage of rows with weight zero",np.sum(train_data.weight==0)/len(train_data))

### 4.3 Distribution of resp

In [ ]:
distribution1('resp',color='green')

In [ ]:
print("Number of rows with resp zero:",np.sum(train_data.resp==0))
print("Percentage of rows with resp zero",np.sum(train_data.resp==0)/len(train_data))

One might find this one thing mentioned in the [data](https://www.kaggle.com/c/jane-street-market-prediction/data) section is confusing.<br/>

"Trades with weight = 0 were intentionally included in the dataset for completeness, although such trades will not contribute towards the scoring evaluation."<br/>

As you can see there are many rows that has weight 0<br/>

Do we have to remove those rows while training ? 🤷‍♀️<br/>

Answer: 
Maybe we can use it while training because we are predicting resp (or action) value.<br/>
but while calculating utility score for validation we should be carefull that<br/>
there are not so many values with weight zero because as we see that in metrics <br/>
pi is ∑j(weightij∗respij∗actionij) so 0 weight will not<br/>
contribute in utility score.

## 4.4 Understanding Metrics 📏📐

One thing is clear that negative resp values contribute lowers the utility score and vice-verca.<br/>
so we need to make a model that gives resp values for the feature but<br/>
what should be the action value for the negative and positive value of resp let's try to understand that<br/>

Lets make a function which make n% correct prediction i.e score 'v' for n% positive resp values and score 0 for <br/>
n% negative resp values and see what are utility scores.

In [ ]:
def utility_score(df,score=0.5,testing=False,true_percentage=0.8):
    if testing:
        df['action'] = 0
        
        false_percentage = 1 - true_percentage
        mask1 = df['resp'] >= 0
        mask2 = df['resp'] < 0
        
        df.loc[mask1,'action'] = score
        
        #make incorrect prediction on the positive resp 
        change_idx = df[mask1].sample(frac=false_percentage).index
        df.loc[change_idx,'action'] = 0
        
        #make incorrect prediction on the negative resp 
        change_idx = df[mask2].sample(frac=false_percentage).index
        df.loc[change_idx,'action'] = score
        
        df['pj'] = df['action'] * df['resp'] * df['weight']
        pi = df[['date','pj']].groupby('date').agg({'pj':['sum']})
        pi.columns = ['pi']
        pi['pi_sq'] = pi['pi']**2
        t = np.sum(pi['pi'])/np.sqrt(np.sum(pi['pi_sq'])) * np.sqrt(250/len(pi['pi'])) 
        u = min(max(t,0),6) * np.sum(pi['pi'])
        
        return u
    
    else:
        df['pj'] = df['action'] * df['resp'] * df['weight']
        pi = df[['date','pj']].groupby('date').agg({'pj':['sum']})
        pi.columns = ['pi']
        pi['pi_sq'] = pi['pi']**2
        t = np.sum(pi['pi'])/np.sqrt(np.sum(pi['pi_sq'])) * np.sqrt(250/len(pi['pi'])) 
        u = min(max(t,0),6) * np.sum(pi['pi'])
        
        return u

In [ ]:
train_data['action'] =0
train_data.loc[train_data['resp']>0.0,'action'] = 1
features = [f"feature_{x}" for x in range(130)]

In [ ]:
train_data.head()

### 4.4.1 Utility Score vs action with 80% accuracy for train data

In [ ]:
scores = list()
for i in np.arange(0,1.05,0.05):
    if i == 0:
        score = utility_score(train_data,score=0.01,testing=True) 
    else:
        score = utility_score(train_data,score=i,testing=True)
    scores.append(score)
px.scatter(x=np.arange(0.0,1.05,.05),y=scores)

We can see that there is positive linear relation ship between action and score which is obvious given the linear metrics<br/>

**Note: this graph is just to see relationship of action vs score.<br/>
We can only use values 0 or 1 for the action.**

### 4.4.2 plot utility score vs Accuracy

In [ ]:
scores = list()
for i in np.arange(0,1.05,0.05):
    score = utility_score(train_data,score=1,testing=True,true_percentage=i)
    scores.append(score)
    
px.scatter(x=np.arange(0.0,1.05,.05),y=scores)

You will only get a utility score if your prediction accuracy is higher than 50%.

### 4.4.3 Distribution of pi - (sum(weight * action * resp)) if action is 0.5

In [ ]:
train_data['pj'] = train_data['action'] * train_data['resp'] * train_data['weight']
pi = train_data[['date','pj']].groupby('date').agg({'pj':['sum']})
pi.columns = ['pi']
distribution1('pi','blue',df=pi)
print("sum of pi is",np.sum(pi['pi']))

### 4.5 Heatmap of features.

In [ ]:
corr = train_data[features+['resp','resp_1','resp_2','resp_3','resp_4']].corr()
fig = px.imshow(corr)
fig.show()

Threre are many features which are highly correlated<br/>
I think we should look into some highly correlated features to see their distribution and relation.

I used plotly so that you can zoom into the feature.<br/>
You can see that most of high correlated feature are near the center diagonal line<br/>
Most important patter is in the bottom right corner if you zoom into it you will see that<br/>
There is alternate high and low correlations<br/>

Another interesing one is 18 to 36 feature<br/>

But there is no feature which is highly correlated to the resp value.

### 4.6 Pairplot for highly related features (18 to 39)

In [ ]:
plt.figure(figsize=(10,10))
sns.pairplot(train_data.loc[:10000,[f"feature_{x}" for x in range(18,39)]]);

😆😆😆
Oh man EDA is fun.
I mean look at it, It looks like those optical illusion images<br/>

## 4.7 Let's play with PCA

### 4.7.1 How good are 4 components of pca at seprating data

In [ ]:
def pca_plot1(features,n_components,target,nrows=10**4):
    pca = PCA(n_components=n_components)
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    train_g_pca = pca.fit_transform(train_d[features])

    total_var = pca.explained_variance_ratio_.sum()*100
    labels = {str(i): f"PC {i+1}" for i in range(n_components)}

    fig = px.scatter_matrix(
        train_g_pca,
        dimensions=range(n_components),
        labels=labels,
        title=f"Total explained variance ratio{total_var:.2f}%",
        color=train_d[target].values
    )

    fig.update_traces(diagonal_visible=True,opacity=0.5)
    fig.show()

In [ ]:
pca_plot1([f"feature_{x}" for x in range(0,130)],4,'resp')

In [ ]:
def pca_plot_3d(features,target,nrows=10**4):
    pca = PCA(n_components=3)
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    train_g_pca = pca.fit_transform(train_d[features])

    total_var = pca.explained_variance_ratio_.sum()*100

    fig = px.scatter_3d(
        train_g_pca,x=0,y=1,z=2,
        title=f"Total explained variance ratio{total_var:.2f}%",
        color=train_d[target].values,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )

    fig.show()

In [ ]:
pca_plot_3d([f"feature_{x}" for x in range(0,130)],'resp')

### 4.7.2 Ploting explained variance 

In [ ]:
def plot_exp_var(features,nrows=10**4):
    pca = PCA()
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    pca.fit(train_d[features])
    exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

    fig = px.area(
        x=range(1, exp_var_cumul.shape[0] + 1),
        y=exp_var_cumul,
        labels={"x": "# Components", "y": "Explained Variance"},
    )
    fig.show()

In [ ]:
plot_exp_var([f"feature_{x}" for x in range(0,130)])

## 4.8 Distribution of date 

Distribution of transaction per day

In [ ]:
distribution1('date','red');

It is good to see that there is not much disturbance in number of trades per day<br/>
we do see that number of transaction at the start from 0 to 50 is high.<br/>
There might be some reason for that.<br/>
But there is no information about the month or year the data was produced so it is hard to tell the reason<br/>

## 4.9 Distribution of Resps

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(511)
sns.kdeplot(train_data['resp_1'],color='#4285F4',shade=True,alpha=1)
plt.subplot(512)
sns.kdeplot(train_data['resp_2'],color='#EA4335',shade=True,alpha=1)
plt.subplot(513)
sns.kdeplot(train_data['resp_3'],color='#FBBC05',shade=True,alpha=1)
plt.subplot(514)
sns.kdeplot(train_data['resp_4'],color='#34A853',shade=True,alpha=1)
plt.subplot(515)
sns.kdeplot(train_data['resp'],color='#4285F4',shade=True,alpha=1);

## 4.10 Distplot of some features

I am going to plot distribution plot of those 10 features which has highest difference in mean<br/>
of data with negative resps and positive resps.

In [ ]:
def distribution2(feature):
    sns.distplot(train_data.loc[train_data['resp']>0,feature],label='negative resp')
    sns.distplot(train_data.loc[train_data['resp']<0,feature],label='positive resp')
    plt.legend()

In [ ]:
temp = train_data.groupby('action')[[f"feature_{x}" for x in range(130)]].mean()
diff_mean = (temp.iloc[1] - temp.loc[0]).abs()
highest_diff_in_mean = diff_mean.sort_values(ascending=False)[:10].index.tolist()

In [ ]:
plt.figure(figsize=(20,15))
for i,feature in enumerate(highest_diff_in_mean):
    plt.subplot(2,5,i+1)
    distribution2(feature)

Even though these features have highest difference in mean of their resp.<br/>
it is difficult to see any difference in their graphs so it is difficult to seprate<br/>
data using few features so we will need many features to make model<br/>

In all the dataset that I have seen there is atleast one feature that you can see its distplot <br/>
and tell that there is a difference but this dataset is different.

And almost every feature have mean value around zero which I think makes it easier for us<br/>
as data would not require any extra preprocessing.

## 4.11 Count of neg and pos resps in train_data

In [ ]:
px.bar(train_data['action'].value_counts(),color=['yellow','blue'],labels=['negative resp','positive resp'])

Number of datapoints in negative and positive resps are almost same which is a good news<br/>
we do not have to worry about training on unbalanced dataset. 

## 4.12 Distplot of weight with neg and pos resps

In [ ]:
distribution2('weight')

## 4.13 Mean Value of some features over time

In [ ]:
feature_df = train_data[features+['date']].groupby('date').mean()
colors = ['#8ECAE6','#219EBC','#023047','#023047','#023047','#0E402D','#023047','#023047','#F77F00','#D62828']

plt.figure(figsize=(20,20))
for i in range(0,10):
    plt.subplot(10,1,i+1)
    ax = sns.lineplot(x=feature_df.index,y=feature_df[f'feature_{i}'],color=colors[i],lw=1)
plt.show()

## 4.14 Mean Value of resp features over time

In [ ]:
plt.figure(figsize=(15,5))
resp_date = train_data[['resp','date']].groupby('date').mean()
sns.lineplot(x=[0,500],y=[0,0],lw=1.5)
sns.lineplot(x=resp_date.index,y=resp_date['resp'],color='#dc2f02',lw=1);

## 4.15 Rolling mean of Mean Value of resp features over time

In [ ]:
resp_date['rolling'] = resp_date['resp'].rolling(10).mean()
plt.figure(figsize=(15,5))
sns.lineplot(x=[0,500],y=[0,0],lw=1.5)
sns.lineplot(x=resp_date.index,y=resp_date['rolling'],color='#dc2f02',lw=1);

## 4.16 Value of resp over 10 random days

In [ ]:
dates = np.random.randint(low=0,high=500,size=10)
colors = ['#8ECAE6','#219EBC','#023047','#023047','#023047','#0E402D','#023047','#023047','#F77F00','#D62828']
plt.figure(figsize=(20,20))
for i,date in enumerate(dates):
    plt.subplot(10,1,i+1)
    sns.lineplot(data=train_data.loc[train_data.date==date,'resp'],lw=1,color=colors[i])
    plt.ylabel(date)
plt.show()

## 4.17 White Noise Time Series Analysis

**What is white noise ? **<br/>
white noise in time series is random number which can not be predicted.<br/>
We can make model for the data if data is not white noise.<br/>

Every signal consist of some form of white noise.<br/>
y(t) = signal + noise.

White noise are series of variable which are independent and they have same variance.<br/>
that means the value in present has no relation with value of past or future.<br/>

If your signal is just white noise it means you can not model it they are just random numbers.<br/>

When a model makes prediction all the errors made by models are just white noise which means.<br/>
model has used all the signal available to make the prediction and all that left is white noise.<br/>

Time series is not white noise if following conditions are satisfied.<br/>
1. Mean is not-zero or near zero.
2. Mean change over time
3. Variance change over time
4. Values correlate with lag values


### 4.17.1 Correlation with lag values.

To check that the values are correlated with lag values we will use pandas autocorrelation function.

In [ ]:
plt.style.use('ggplot')
sample = train_data.resp[:10000]
plt.figure(figsize=(15,7))
pd.plotting.autocorrelation_plot(sample,lw=1);

Acctually looking at it. I looks like white noise.<br/>
straight grey line is 95% confidence bar.<br/>
dashed grey line is 99% confidence bar.<br/>

There are few points which are crossing that line.<br/>
So is the data just white noise or did I missed something.<br/>

## LGBM Model

We are going to make a baseline model which is simple classification model.<br/>

## PurgedGroupTimeSeriesSplit¶


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
features = [f'feature_{x}' for x in range(130)] 

target_feature = 'action'

train_mean = train_data.mean()

train_data.fillna(train_mean,inplace=True)
train_data['action'] = 0
train_data.loc[train_data['resp']>0.0,'action'] = 1

dates = train_data['date'].to_numpy()

target = train_data[target_feature].to_numpy()
train_data = train_data[features].to_numpy()

In [ ]:
params = {
    'objective': 'binary',
    'metrics':['auc'],
}

nfolds=3
group_gap = 31

In [ ]:
kfold = PurgedGroupTimeSeriesSplit(n_splits=nfolds,group_gap=group_gap)
lgb_models = list()

for k ,(train_idx,valid_idx) in enumerate(kfold.split(train_data,target.ravel(),dates.ravel())): 
    
    lgb_train = lgb.Dataset(train_data[train_idx],target[train_idx].ravel())
    lgb_valid = lgb.Dataset(train_data[valid_idx],target[valid_idx].ravel())
    
    model = lgb.train(
        params,
        lgb_train,
        valid_sets = [lgb_train,lgb_valid],
        num_boost_round = 10000,
        verbose_eval = 50,
        early_stopping_rounds = 10,
    )
    
    lgb_models.append(model)

## Feature Importance

In [ ]:
def plot_feature_importance(model):
    feature_importance = pd.DataFrame({"feature":features,"importance":model.feature_importance(importance_type='gain')})
    feature_importance = feature_importance.sort_values(by='importance',ascending=False)
    
    plt.figure(figsize=(20,20))
    sns.barplot(data=feature_importance,x='importance',y='feature')
    plt.title("Importance type Gain")
    
    for idx, v in enumerate(feature_importance.importance):
            plt.text(v, idx, "  {:.2e}".format(v))

In [ ]:
plot_feature_importance(lgb_models[0])

In [ ]:
predictions = np.zeros(test_data.shape[0])

for model in  lgb_models:
    predictions += model.predict(test_data[features])
    
predictions /= len(lgb_models)
sns.distplot(predictions);

In [ ]:
del train_data, target
gc.collect()

## Make Prediction

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
%%time
for (test_df, sample_prediction_df) in iter_test:
    
    prediction = 0
    for model in lgb_models:
        prediction += model.predict(test_df[features])[0]
    
    prediction /= len(lgb_models)
    prediction = prediction > 0.5
    sample_prediction_df.action = prediction.astype(int)
    env.predict(sample_prediction_df)

In [ ]:
# env.predict(sample)

In [ ]:
submission = pd.read_csv('./submission.csv')
submission.head()

In [ ]:
sns.countplot(submission.action);